### Import Dependencies

In [39]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean
import os
import geojson

### Calculate Geometric Median for precincts in LA

In [3]:
# https://stackoverflow.com/questions/30299267/geometric-median-of-multidimensional-points
# https://www.pnas.org/content/pnas/97/4/1423.full.pdf

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [24]:
def jsonFileLoader(file, year):
    #read json to df and transpose
    crime_df = pd.read_json(file).transpose()

    #group by precinct 
    gb = crime_df.groupby('LAPD_Area')    
    split_crime_df = [gb.get_group(x) for x in gb.groups]

    list1 = []
    
    for i, e in enumerate(split_crime_df):
        prec_crime_np = split_crime_df[i][['LAT','LON']].to_numpy()
        
        y = list(geometric_median(prec_crime_np))
        x = str(split_crime_df[i]["LAPD_Area_Name"].values[0])
        
        geoJsonE = {}
        geoJsonE["type"] = "Feature"
        geoJsonE["geometry"] = {"type":"Point","coordinates":[y[1], y[0]]}
        geoJsonE["properties"] = {"name":x, "year":year}
        
        list1.append(geoJsonE)
    return list1


In [43]:
year = [2013,2014]#,2015,2016,2017,2018,2019]
filePath = '../static/data/'
fileName = f'crime_median'
outputFile = f"{filePath}{fileName}.geojson"

geoJson = {}
listGeo = []
geoJson["type"] = "FeatureCollection"  

for y in year:    
    file_to_load = f'{filePath}source/crime_{y}.json'

    listGeo.append(jsonFileLoader(file_to_load, y))

f = open(outputFile, "w")
geoJson['features'] = listGeo
f.write(str(geojson.dumps(geoJson)))
f.close()

In [42]:
print(geojson.dumps(geoJson))

{"type": "FeatureCollection", "features": [[{"type": "Feature", "geometry": {"type": "Point", "coordinates": [-118.25044004338203, 34.046268430317454]}, "properties": {"name": "Central", "year": 2013}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-118.27332504902962, 34.06107176670108]}, "properties": {"name": "Rampart", "year": 2013}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-118.31555108026427, 34.01838682642745]}, "properties": {"name": "Southwest", "year": 2013}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-118.20472845179633, 34.05261478030283]}, "properties": {"name": "Hollenbeck", "year": 2013}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-118.28659168098801, 33.764045318683266]}, "properties": {"name": "Harbor", "year": 2013}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-118.32996665828792, 34.09982065823611]}, "properties": {"name": "Hollywood", "year": 2013}}, 